# Importing libraries

In [2]:
import os, glob, json, csv, subprocess, sys, re
from urllib.request import urlopen
from itertools import chain
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Counter
import pandas as pd
import plotly.graph_objs as go
import plotly
from plotly.offline import plot, init_notebook_mode, iplot
import itertools

%matplotlib inline

# Defining directories

In [3]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/hbase/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Defining git command function

In [4]:
def execute_shell_command(buglink, id1, id2, cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return buglink, id1, id2, out, error
    pipe.wait()

# Load data from CSV file

In [10]:
dfhist = pd.read_csv(analyze_dir + '03_file-diff/diffhist-bugs.csv')
dfmin = pd.read_csv(analyze_dir + '03_file-diff/diffmin-bugs.csv')
dfmyers = pd.read_csv(analyze_dir + '03_file-diff/diffmyers-bugs.csv')
dfpat = pd.read_csv(analyze_dir + '03_file-diff/diffpat-bugs.csv')

Defining columns

In [11]:
cols = ['bug_id','bugintro_commitID','parent_id','#insertions','#deletions','filename','#line_changed']
column = ['bug_id','bugintro_commitID','parent_id','filename','#deletions']

# Removing files without buglines

In [12]:
dthist = dfhist[cols][dfhist[cols]['#deletions'] != 0]
dtmin = dfmin[cols][dfmin[cols]['#deletions'] != 0]
dtmyers = dfmyers[cols][dfmyers[cols]['#deletions'] != 0]
dtpat = dfpat[cols][dfpat[cols]['#deletions'] != 0]
dtpat

,bug_id,bugintro_commitID,parent_id,#insertions,#deletions,filename,#line_changed
0,HBASE-9971,8a6291a896,c9d4d2c9b5e30ae79d0ec7782709eaa917600f9a,4,5,src/main/java/org/apache/hadoop/hbase/client/H...,9
1,HBASE-9961,18e018b3d9,16b799dae56b54f82a14dc0e0ad24977f06df6f6,17,13,hbase-client/src/main/java/org/apache/hadoop/h...,30
3,HBASE-9958,c5a3048fec,feca3e26c468744c91a90ecd4a972a5bd114526c,51,31,hbase-client/src/main/java/org/apache/hadoop/h...,82
4,HBASE-9958,c5a3048fec,feca3e26c468744c91a90ecd4a972a5bd114526c,5,5,hbase-client/src/main/java/org/apache/hadoop/h...,10
5,HBASE-9958,c5a3048fec,feca3e26c468744c91a90ecd4a972a5bd114526c,1,8,hbase-client/src/main/java/org/apache/hadoop/h...,9
6,HBASE-9958,c5a3048fec,feca3e26c468744c91a90ecd4a972a5bd114526c,10,2,hbase-client/src/main/java/org/apache/hadoop/h...,12
8,HBASE-9908,ff78f217fa,bf5f9e5240847c8788c9dce700dd94750325863d,1,1,hbase-common/src/main/java/org/apache/hadoop/h...,2
10,HBASE-9908,ff78f217fa,bf5f9e5240847c8788c9dce700dd94750325863d,4,2,hbase-common/src/test/java/org/apache/hadoop/h...,6
11,HBASE-9908,ff78f217fa,bf5f9e5240847c8788c9dce700dd94750325863d,1,3,hbase-common/src/test/java/org/apache/hadoop/h...,4
12,HBASE-9908,ff78f217fa,bf5f9e5240847c8788c9dce700dd94750325863d,1,1,hbase-server/src/main/java/org/apache/hadoop/h...,2


In [14]:
print ("Number of data <histogram>: %i" %(len(dthist)))
print ("Number of data <minimal>: %i" %(len(dtmin)))
print ("Number of data <myers>: %i" %(len(dtmyers)))
print ("Number of data <patience>: %i" %(len(dtpat)))

Number of data <histogram>: 2599
Number of data <minimal>: 2599
Number of data <myers>: 2599
Number of data <patience>: 2599


List of filename with deleted lines is saved into CSV file

In [16]:
dt_algorithm = [dthist, dtmin, dtmyers, dtpat]
algorithms = ['histogram','minimal','myers','patience']

In [19]:
for n, dt in enumerate(dt_algorithm):
    d_alg = []
    for c in range(0, len(dt)):
        sys.stdout.write("\rWriting data for algorithm {}, no: {} / {}".format((n+1), (c+1), len(dt)))
        sys.stdout.flush()
        item = dt.iloc[c]
        tem = [item[0],item[1],item[2],item[5],item[4]]
        d_alg.append(tem)
        
    #save into CSV file:
    with open(analyze_dir + '03_file-diff/01_filename/' + algorithms[n] + '_files.csv','w') as csvfile:
        writers = csv.writer(csvfile, delimiter=",")
        writers.writerow(column)
        for zz in d_alg:
            writers.writerow(zz)
    print(" ==> File '" + algorithms[n] + "_files.csv' has been created")

Writing data for algorithm 1, no: 2599 / 2599 ==> File 'histogram_files.csv' has been created
Writing data for algorithm 2, no: 2599 / 2599 ==> File 'minimal_files.csv' has been created
Writing data for algorithm 3, no: 2599 / 2599 ==> File 'myers_files.csv' has been created
Writing data for algorithm 4, no: 2599 / 2599 ==> File 'patience_files.csv' has been created


# Summarize total buglines for each algorithm

In [20]:
fname = pd.read_csv(analyze_dir + "03_file-diff/01_filename/histogram_files.csv")
fname = fname[column]
summ = fname['#deletions'].sum()
summ

124222

In [34]:
total_bugline = []
n = 0
for file in glob.iglob(analyze_dir + "03_file-diff/01_filename/*", recursive=True):
    fname = pd.read_csv(file)
    fname = fname[column]
    sum_bugs = fname['#deletions'].sum()
    file = re.search("\/(\w+)\_files", file)
    file = file.groups()
    temp = [file[0], sum_bugs]
    total_bugline.append(temp)
    n += 1
total_bugline.sort()
total_bugline

[['histogram', 124222],
 ['minimal', 123981],
 ['myers', 124338],
 ['patience', 124135]]

In [35]:
with open(analyze_dir + "03_file-diff/01_filename/total_bugline.csv","w") as myfile:
    cols = ['diff_algorithm','total_bugline']
    writers = csv.writer(myfile, delimiter=",")
    writers.writerow(cols)
    for ss in total_bugline:
        writers.writerow(ss)
print ("File 'total_bugline' has been created")

File 'total_bugline' has been created


In [36]:
dftotal = pd.read_csv(analyze_dir + "03_file-diff/01_filename/total_bugline.csv")
dftotal = dftotal[cols]
dftotal

,diff_algorithm,total_bugline
0,histogram,124222
1,minimal,123981
2,myers,124338
3,patience,124135


In [38]:
graphtotal = go.Bar(
    x = dftotal['diff_algorithm'],
    y = dftotal['total_bugline'],
    name = dftotal['diff_algorithm'],
    text = dftotal['total_bugline'],
    textposition = 'auto',
    marker = dict(
        color = 'rgb(150,255,200)',
        line = dict(
            color='rgb(8,48,107)',
            width=1.5),
    ),
    opacity=0.6
)

dt = [graphtotal]
layout = go.Layout(
    title = "Total number of bugline for each algorithm"
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)